# Module 04: Clock Synchronization in Distributed Systems

**Difficulty**: ⭐⭐⭐  
**Estimated Time**: 75 minutes  
**Prerequisites**: 
- [Module 00: Setup and Introduction](00_setup_introduction.ipynb)
- [Module 01: Introduction to Distributed Systems](01_introduction_distributed_systems.ipynb)
- [Module 02: Transparency and Architectures](02_transparency_architectures.ipynb)
- [Module 03: Memory Management](03_memory_management.ipynb)

## Learning Objectives

By the end of this notebook, you will be able to:
1. **Explain** why clock synchronization is critical in distributed systems
2. **Differentiate** between physical clocks and logical clocks
3. **Implement** Cristian's Algorithm for clock synchronization
4. **Implement** Berkeley Algorithm for clock synchronization
5. **Understand** Network Time Protocol (NTP) basics
6. **Apply** Lamport Timestamps for event ordering
7. **Use** Vector Clocks to capture causality
8. **Analyze** clock synchronization accuracy and performance

---

## Section 1: Why Clock Synchronization?

### The Problem

In a distributed system, each computer has its own **physical clock**. These clocks:
- Run at slightly different rates (**clock drift**)
- May be set to different times (**clock skew**)
- Are affected by temperature, voltage, manufacturing variations

### Why It Matters

**Examples where synchronization is critical:**

1. **Make commands**: Build system needs to know if source files are newer than compiled files
2. **Distributed databases**: Ordering of transactions matters
3. **Financial trading**: Order of trades affects market prices
4. **Logging/Debugging**: Need to correlate events across multiple servers
5. **Expiring credentials**: Security tokens with expiration times

### Real-World Impact

**Example**: Amazon Web Services recommends clock synchronization within 1ms for distributed systems. Without it:
- Database replicas can become inconsistent
- Security protocols can fail
- Logs become impossible to correlate

In [ ]:
# Setup: Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from collections import deque
from dataclasses import dataclass
from typing import List, Tuple

# Configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
random.seed(42)

print("✓ Environment ready for clock synchronization simulations")

### 1.1 Clock Drift Demonstration

Let's simulate how clocks drift apart over time.

In [ ]:
def simulate_clock_drift(num_servers=4, duration=100, drift_rates=None):
    """
    Simulate clock drift on multiple servers.
    
    Parameters:
    -----------
    num_servers : int
        Number of servers to simulate
    duration : int
        Simulation duration in seconds
    drift_rates : list
        Drift rates in ppm (parts per million). If None, random values used.
        Positive = clock runs fast, Negative = clock runs slow
    """
    if drift_rates is None:
        # Typical quartz crystal drift: ±50 ppm
        drift_rates = np.random.uniform(-50, 50, num_servers)
    
    time_points = np.arange(0, duration + 1)
    server_times = np.zeros((num_servers, len(time_points)))
    
    # Calculate clock time for each server at each time point
    for i in range(num_servers):
        # drift_rate in ppm means: for every 1,000,000 seconds,
        # the clock gains/loses drift_rate seconds
        drift_per_second = drift_rates[i] / 1_000_000
        server_times[i] = time_points * (1 + drift_per_second)
    
    # Visualize
    plt.figure(figsize=(14, 8))
    
    colors = plt.cm.tab10(np.linspace(0, 1, num_servers))
    
    for i in range(num_servers):
        plt.plot(time_points, server_times[i], 
                label=f'Server {i} ({drift_rates[i]:+.1f} ppm)',
                color=colors[i], linewidth=2)
    
    # Reference: perfect clock
    plt.plot(time_points, time_points, 'k--', linewidth=2, 
            label='Perfect Clock (no drift)', alpha=0.5)
    
    plt.xlabel('Real Time (seconds)', fontsize=12)
    plt.ylabel('Clock Reading (seconds)', fontsize=12)
    plt.title('Clock Drift in Distributed Systems\n(Different servers drift at different rates)', 
             fontsize=14, fontweight='bold')
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate maximum divergence
    final_divergence = np.max(server_times[:, -1]) - np.min(server_times[:, -1])
    
    print(f"\n📊 Clock Drift Analysis:")
    print(f"   Duration: {duration} seconds")
    print(f"   Maximum divergence: {final_divergence:.6f} seconds")
    print(f"   That's {final_divergence * 1000:.2f} milliseconds!")
    print(f"\n   Over 1 day (86,400 sec), clocks could diverge by: {(final_divergence / duration) * 86400:.2f} seconds")

# Run simulation
simulate_clock_drift(num_servers=4, duration=100)

---

## Section 2: Physical Clock Synchronization

### 2.1 Cristian's Algorithm

**Cristian's Algorithm** (1989) is a simple method for synchronizing clocks in a client-server architecture.

#### How It Works:

```
CLIENT                           TIME SERVER
  |                                   |
  |  (1) Request (T0)                 |
  |---------------------------------->|
  |                                   | (2) Server responds with Ts
  |         Response (Ts, T1)         |
  |<----------------------------------|
  |
  | (3) Estimate network delay = (T1-T0)/2
  | (4) Set clock to: Ts + delay
```

**Key Assumptions:**
- Network delay is symmetric (request time ≈ response time)
- Time server's clock is accurate
- Round-trip time is measured

#### Algorithm Steps:

1. Client notes current time T₀
2. Client sends request to time server
3. Server responds immediately with its time Tₛ
4. Client receives response at time T₁
5. Client estimates network delay: δ = (T₁ - T₀) / 2
6. Client sets clock to: Tₛ + δ

In [ ]:
class TimeServer:
    """
    Simulates an accurate time server.
    """
    def __init__(self):
        self.current_time = 0.0  # Server's accurate time
    
    def get_time(self):
        """Return current time (assume this is accurate)."""
        return self.current_time
    
    def advance_time(self, delta):
        """Advance server time by delta seconds."""
        self.current_time += delta


class ClientClock:
    """
    Simulates a client with an inaccurate clock.
    """
    def __init__(self, initial_offset=0.0, drift_rate_ppm=0.0):
        self.offset = initial_offset  # How far off from true time
        self.drift_rate_ppm = drift_rate_ppm  # Clock drift in ppm
        self.true_time = 0.0
    
    def get_time(self):
        """Return client's clock reading (may be inaccurate)."""
        drift = self.true_time * (self.drift_rate_ppm / 1_000_000)
        return self.true_time + self.offset + drift
    
    def set_time(self, new_time):
        """Set client clock to new time."""
        self.offset = new_time - self.true_time
    
    def advance_time(self, delta):
        """Advance true time by delta seconds."""
        self.true_time += delta


def cristians_algorithm(client, server, network_delay_range=(0.01, 0.05)):
    """
    Implement Cristian's Algorithm for clock synchronization.
    
    Parameters:
    -----------
    client : ClientClock
        Client clock to synchronize
    server : TimeServer
        Accurate time server
    network_delay_range : tuple
        Min and max network delay in seconds
    
    Returns:
    --------
    dict : Synchronization details
    """
    # Step 1: Client notes time before request
    T0 = client.get_time()
    
    # Simulate network delay (request travel time)
    request_delay = random.uniform(*network_delay_range)
    
    # Step 2: Server receives request and responds with its time
    # Advance time by request delay
    client.advance_time(request_delay)
    server.advance_time(request_delay)
    Ts = server.get_time()
    
    # Simulate network delay (response travel time)
    response_delay = random.uniform(*network_delay_range)
    client.advance_time(response_delay)
    server.advance_time(response_delay)
    
    # Step 3: Client receives response and notes time
    T1 = client.get_time()
    
    # Step 4: Calculate estimated network delay
    round_trip_time = T1 - T0
    estimated_delay = round_trip_time / 2
    
    # Step 5: Set client clock
    new_time = Ts + estimated_delay
    client.set_time(new_time)
    
    # Calculate accuracy
    actual_delay = request_delay + response_delay
    true_server_time_now = server.get_time()
    client_time_after_sync = client.get_time()
    sync_error = abs(client_time_after_sync - true_server_time_now)
    
    return {
        'T0': T0,
        'Ts': Ts,
        'T1': T1,
        'round_trip_time': round_trip_time,
        'estimated_delay': estimated_delay,
        'actual_delay': actual_delay,
        'sync_error': sync_error,
        'request_delay': request_delay,
        'response_delay': response_delay
    }


# Demonstrate Cristian's Algorithm
print("=" * 70)
print("CRISTIAN'S ALGORITHM DEMONSTRATION")
print("=" * 70)

# Create time server and client
server = TimeServer()
client = ClientClock(initial_offset=5.0)  # Client clock is 5 seconds ahead

print(f"\nInitial State:")
print(f"   Server time: {server.get_time():.6f} seconds")
print(f"   Client time: {client.get_time():.6f} seconds")
print(f"   Client offset: {client.get_time() - server.get_time():.6f} seconds")

# Run synchronization
print(f"\n🔄 Running Cristian's Algorithm...\n")
result = cristians_algorithm(client, server)

print(f"Synchronization Details:")
print(f"   T0 (request time): {result['T0']:.6f} seconds")
print(f"   Ts (server time): {result['Ts']:.6f} seconds")
print(f"   T1 (response time): {result['T1']:.6f} seconds")
print(f"   Round-trip time: {result['round_trip_time']:.6f} seconds")
print(f"   Estimated delay: {result['estimated_delay']:.6f} seconds")
print(f"   Actual delay: {result['actual_delay']:.6f} seconds")
print(f"\n   Synchronization error: {result['sync_error']:.6f} seconds ({result['sync_error'] * 1000:.2f} ms)")

print(f"\nFinal State:")
print(f"   Server time: {server.get_time():.6f} seconds")
print(f"   Client time: {client.get_time():.6f} seconds")
print(f"   Remaining offset: {client.get_time() - server.get_time():.6f} seconds")

if result['sync_error'] < 0.001:
    print(f"\n✅ Excellent synchronization! Error < 1 ms")
elif result['sync_error'] < 0.010:
    print(f"\n✅ Good synchronization! Error < 10 ms")
else:
    print(f"\n⚠️  Synchronization error is significant")

### 2.2 Analyzing Cristian's Algorithm Accuracy

Let's test the algorithm multiple times to see how network variability affects accuracy.

In [ ]:
def test_cristians_algorithm(num_trials=50, initial_offset=10.0):
    """
    Test Cristian's Algorithm multiple times to analyze accuracy.
    """
    errors = []
    round_trip_times = []
    
    for _ in range(num_trials):
        server = TimeServer()
        client = ClientClock(initial_offset=initial_offset)
        result = cristians_algorithm(client, server)
        
        errors.append(result['sync_error'] * 1000)  # Convert to milliseconds
        round_trip_times.append(result['round_trip_time'] * 1000)
    
    # Visualize results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot 1: Synchronization errors
    ax1.hist(errors, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    ax1.axvline(np.mean(errors), color='red', linestyle='--', linewidth=2, 
               label=f'Mean: {np.mean(errors):.2f} ms')
    ax1.set_xlabel('Synchronization Error (milliseconds)', fontsize=11)
    ax1.set_ylabel('Frequency', fontsize=11)
    ax1.set_title('Cristian\'s Algorithm: Synchronization Error Distribution', 
                 fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Plot 2: Error vs Round-Trip Time
    ax2.scatter(round_trip_times, errors, alpha=0.6, color='coral')
    ax2.set_xlabel('Round-Trip Time (milliseconds)', fontsize=11)
    ax2.set_ylabel('Synchronization Error (milliseconds)', fontsize=11)
    ax2.set_title('Error vs Network Delay', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print(f"\n📊 Statistics from {num_trials} synchronization attempts:")
    print(f"   Mean error: {np.mean(errors):.3f} ms")
    print(f"   Std deviation: {np.std(errors):.3f} ms")
    print(f"   Min error: {np.min(errors):.3f} ms")
    print(f"   Max error: {np.max(errors):.3f} ms")
    print(f"   Median error: {np.median(errors):.3f} ms")
    
    print(f"\n💡 Key Insight:")
    print(f"   Cristian's Algorithm accuracy depends on network stability.")
    print(f"   Lower and more consistent round-trip times = better accuracy.")

test_cristians_algorithm(num_trials=50, initial_offset=10.0)

---

## Section 3: Berkeley Algorithm

**Berkeley Algorithm** is different from Cristian's Algorithm:
- **No single time server** acting as the source of truth
- Uses **averaging** across all clocks
- The **master** (coordinator) polls all slaves for their time
- Master calculates average time
- Master tells each slave how to adjust (positive or negative)

### How It Works:

```
MASTER                    SLAVE 1         SLAVE 2         SLAVE 3
  |                          |               |               |
  | (1) Request time         |               |               |
  |------------------------->|               |               |
  |--------------------------|-------------->|               |
  |--------------------------|---------------|-------------->|
  |                          |               |               |
  |     (2) Responses        |               |               |
  |<-------------------------|               |               |
  |<-------------------------|---------------|               |
  |<-------------------------|---------------|---------------|
  |
  | (3) Calculate average time
  | (4) Compute adjustments for each slave
  |
  | (5) Send adjustments      |               |               |
  |------------------------->|               |               |
  |  "Add +2 sec"            |               |               |
  |--------------------------|-------------->|               |
  |  "Subtract -1 sec"       |               |               |
  |--------------------------|---------------|-------------->|
  |  "Add +0.5 sec"          |               |               |
```

### Advantages:
- No single point of failure for time accuracy
- Fault-tolerant (can ignore outliers)
- Works well in LANs where network delays are small and predictable

In [ ]:
@dataclass
class Node:
    """Represents a node in the distributed system."""
    node_id: int
    clock_time: float
    drift_rate: float = 0.0  # ppm
    
    def get_time(self):
        return self.clock_time
    
    def adjust_time(self, adjustment):
        self.clock_time += adjustment


def berkeley_algorithm(nodes: List[Node], master_id: int = 0, 
                       network_delay_range=(0.001, 0.005)):
    """
    Implement Berkeley Algorithm for clock synchronization.
    
    Parameters:
    -----------
    nodes : List[Node]
        List of all nodes including master
    master_id : int
        ID of the master/coordinator node
    network_delay_range : tuple
        Range of network delays
    
    Returns:
    --------
    dict : Synchronization details
    """
    master = nodes[master_id]
    
    print(f"\n🎯 Master Node: {master_id}")
    print(f"\nStep 1: Master polls all nodes for their time\n")
    
    # Step 1: Master requests time from all slaves
    reported_times = []
    for node in nodes:
        # Simulate network delay
        delay = random.uniform(*network_delay_range)
        reported_time = node.get_time() + delay  # Time when master receives response
        reported_times.append(reported_time)
        print(f"   Node {node.node_id}: reports {node.get_time():.6f} s ")
               f"(received at master: {reported_time:.6f} s)")
    
    # Step 2: Master calculates average time
    average_time = np.mean(reported_times)
    print(f"\nStep 2: Master calculates average")
    print(f"   Average time: {average_time:.6f} seconds")
    
    # Step 3: Master computes adjustments
    adjustments = []
    print(f"\nStep 3: Master computes adjustments\n")
    for i, node in enumerate(nodes):
        adjustment = average_time - reported_times[i]
        adjustments.append(adjustment)
        sign = '+' if adjustment >= 0 else ''
        print(f"   Node {node.node_id}: {sign}{adjustment:.6f} seconds")
    
    # Step 4: Master sends adjustments to all nodes
    print(f"\nStep 4: Nodes apply adjustments\n")
    for i, node in enumerate(nodes):
        old_time = node.get_time()
        node.adjust_time(adjustments[i])
        new_time = node.get_time()
        print(f"   Node {node.node_id}: {old_time:.6f} → {new_time:.6f} s")
    
    # Calculate synchronization quality
    final_times = [node.get_time() for node in nodes]
    max_divergence = max(final_times) - min(final_times)
    
    return {
        'reported_times': reported_times,
        'average_time': average_time,
        'adjustments': adjustments,
        'final_times': final_times,
        'max_divergence': max_divergence
    }


# Demonstrate Berkeley Algorithm
print("=" * 70)
print("BERKELEY ALGORITHM DEMONSTRATION")
print("=" * 70)

# Create nodes with different clock times
nodes = [
    Node(node_id=0, clock_time=10.000),  # Master
    Node(node_id=1, clock_time=10.015),
    Node(node_id=2, clock_time=9.985),
    Node(node_id=3, clock_time=10.005),
]

print(f"\nInitial clock times:")
for node in nodes:
    print(f"   Node {node.node_id}: {node.get_time():.6f} seconds")

initial_divergence = max(n.get_time() for n in nodes) - min(n.get_time() for n in nodes)
print(f"\n   Initial divergence: {initial_divergence:.6f} seconds ({initial_divergence*1000:.2f} ms)")

# Run Berkeley Algorithm
result = berkeley_algorithm(nodes, master_id=0)

print(f"\n" + "=" * 70)
print(f"\n✅ Synchronization Complete!")
print(f"\nFinal clock times:")
for node in nodes:
    print(f"   Node {node.node_id}: {node.get_time():.6f} seconds")

print(f"\n   Final divergence: {result['max_divergence']:.6f} seconds "
      f"({result['max_divergence']*1000:.2f} ms)")
print(f"   Improvement: {(initial_divergence - result['max_divergence'])*1000:.2f} ms")

### 3.1 Comparing Cristian's vs Berkeley Algorithm

In [ ]:
# Create comparison visualization
comparison_data = {
    'Aspect': ['Architecture', 'Authority', 'Fault Tolerance', 'Network Traffic', 
               'Accuracy', 'Use Case'],
    'Cristian\'s Algorithm': [
        'Client-Server',
        'Single time server',
        'Low (server is SPOF)',
        'Lower (1 request + 1 response per client)',
        'Good for small networks',
        'When accurate time server available'
    ],
    'Berkeley Algorithm': [
        'Peer-to-peer',
        'Distributed (averaging)',
        'High (no SPOF)',
        'Higher (N requests + N responses)',
        'Good for LANs',
        'When no single accurate clock available'
    ]
}

fig, ax = plt.subplots(figsize=(14, 8))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=[comparison_data['Aspect'],
                           comparison_data['Cristian\'s Algorithm'],
                           comparison_data['Berkeley Algorithm']],
                rowLabels=['Aspect', 'Cristian\'s', 'Berkeley'],
                cellLoc='left',
                loc='center',
                colWidths=[0.15, 0.28, 0.28, 0.28, 0.28, 0.28])

table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2.5)

# Color header row
for i in range(len(comparison_data['Aspect'])):
    table[(0, i)].set_facecolor('#4ECDC4')
    table[(0, i)].set_text_props(weight='bold')

# Color algorithm rows
for i in range(len(comparison_data['Aspect'])):
    table[(1, i)].set_facecolor('#FFE66D')
    table[(2, i)].set_facecolor('#FF6B6B')

plt.title('Clock Synchronization Algorithms Comparison', 
         fontsize=14, fontweight='bold', pad=20)
plt.show()

print("\n💡 Key Takeaway:")
print("   Choose Cristian's when you have a reliable time server (e.g., NTP server).")
print("   Choose Berkeley when you want fault tolerance and no single clock is trusted.")

---

## Section 4: Logical Clocks

Sometimes we don't need to know the **exact time**, we just need to know the **order of events**.

### 4.1 Lamport Timestamps

**Lamport Timestamps** (1978) provide a way to order events in a distributed system without synchronized physical clocks.

#### Rules:

1. Each process maintains a **logical clock** (counter)
2. Before executing an event, increment the clock: `LC = LC + 1`
3. When sending a message, include the current logical clock
4. When receiving a message:
   - `LC = max(LC, message_timestamp) + 1`

#### Key Property:

If event A happens before event B, then `timestamp(A) < timestamp(B)`

**Note**: The converse is not always true! (False causality)

In [ ]:
class LamportClock:
    """
    Implements Lamport's Logical Clock.
    """
    def __init__(self, process_id):
        self.process_id = process_id
        self.clock = 0
        self.events = []  # Log of events
    
    def local_event(self, description):
        """Process a local event."""
        self.clock += 1
        self.events.append({
            'type': 'local',
            'timestamp': self.clock,
            'description': description
        })
        return self.clock
    
    def send_message(self, description):
        """Send a message to another process."""
        self.clock += 1
        timestamp = self.clock
        self.events.append({
            'type': 'send',
            'timestamp': timestamp,
            'description': description
        })
        return timestamp
    
    def receive_message(self, message_timestamp, description):
        """Receive a message from another process."""
        # Update clock to max(local_clock, message_timestamp) + 1
        self.clock = max(self.clock, message_timestamp) + 1
        self.events.append({
            'type': 'receive',
            'timestamp': self.clock,
            'description': description,
            'message_timestamp': message_timestamp
        })
        return self.clock
    
    def get_time(self):
        return self.clock
    
    def print_log(self):
        print(f"\n📋 Process {self.process_id} Event Log:")
        for event in self.events:
            if event['type'] == 'local':
                print(f"   [{event['timestamp']}] LOCAL: {event['description']}")
            elif event['type'] == 'send':
                print(f"   [{event['timestamp']}] SEND: {event['description']}")
            elif event['type'] == 'receive':
                print(f"   [{event['timestamp']}] RECEIVE: {event['description']} "
                      f"(msg timestamp: {event['message_timestamp']})")


# Demonstrate Lamport Timestamps
print("=" * 70)
print("LAMPORT TIMESTAMPS DEMONSTRATION")
print("=" * 70)

# Create three processes
P1 = LamportClock(process_id=1)
P2 = LamportClock(process_id=2)
P3 = LamportClock(process_id=3)

# Simulate events
print("\n🎬 Simulating distributed system events...\n")

# P1 does local work
P1.local_event("Initialize database")
print(f"P1: Local event → Clock = {P1.get_time()}")

P1.local_event("Read config file")
print(f"P1: Local event → Clock = {P1.get_time()}")

# P1 sends message to P2
msg_timestamp = P1.send_message("Request data from P2")
print(f"P1: Send message to P2 → Clock = {P1.get_time()}")

# P2 does local work
P2.local_event("Start processing")
print(f"P2: Local event → Clock = {P2.get_time()}")

# P2 receives message from P1
P2.receive_message(msg_timestamp, "Received request from P1")
print(f"P2: Receive message from P1 (ts={msg_timestamp}) → Clock = {P2.get_time()}")

# P2 sends message to P3
msg_timestamp2 = P2.send_message("Forward request to P3")
print(f"P2: Send message to P3 → Clock = {P2.get_time()}")

# P3 does local work
P3.local_event("Initialize")
print(f"P3: Local event → Clock = {P3.get_time()}")

# P3 receives message from P2
P3.receive_message(msg_timestamp2, "Received request from P2")
print(f"P3: Receive message from P2 (ts={msg_timestamp2}) → Clock = {P3.get_time()}")

# P3 sends response to P1
msg_timestamp3 = P3.send_message("Send data to P1")
print(f"P3: Send message to P1 → Clock = {P3.get_time()}")

# P1 receives response
P1.receive_message(msg_timestamp3, "Received data from P3")
print(f"P1: Receive message from P3 (ts={msg_timestamp3}) → Clock = {P1.get_time()}")

# Print event logs
P1.print_log()
P2.print_log()
P3.print_log()

print("\n💡 Observation:")
print("   Lamport timestamps ensure that if event A causes event B,")
print("   then timestamp(A) < timestamp(B).")
print("   This gives us a partial ordering of events!")

---

## Section 5: Exercises

### Exercise 1: Implement NTP-style Synchronization

**Task**: Implement a simplified version of the Network Time Protocol (NTP) that:
1. Measures round-trip time
2. Calculates offset and delay
3. Uses multiple samples to filter out outliers

**Hints**:
- NTP uses 4 timestamps: T1, T2, T3, T4
- Offset = ((T2 - T1) + (T3 - T4)) / 2
- Delay = (T4 - T1) - (T3 - T2)

In [ ]:
def ntp_synchronization(client_clock, server_clock, num_samples=8):
    """
    EXERCISE: Implement NTP-style clock synchronization.
    
    Parameters:
    -----------
    client_clock : ClientClock
        Client clock to synchronize
    server_clock : TimeServer
        Accurate time server
    num_samples : int
        Number of samples to take (NTP typically uses 8)
    
    Returns:
    --------
    dict : Synchronization results
    
    TODO: Implement this function
    """
    # YOUR CODE HERE
    pass

# Test your implementation
# server = TimeServer()
# client = ClientClock(initial_offset=5.0)
# result = ntp_synchronization(client, server)
# print(f"Synchronization error: {result['error']:.6f} seconds")

### Exercise 2: Detect Clock Drift

**Task**: Implement a monitoring system that:
1. Periodically synchronizes clocks
2. Detects when drift exceeds a threshold
3. Triggers re-synchronization automatically

**Requirements**:
- Monitor 5 servers
- Check every 10 simulated seconds
- Threshold: 100 milliseconds

In [ ]:
class ClockMonitor:
    """
    EXERCISE: Implement clock drift monitoring and auto-synchronization.
    
    TODO: Complete this class
    """
    def __init__(self, num_servers=5, drift_threshold_ms=100):
        # YOUR CODE HERE
        pass
    
    def check_drift(self):
        """Check if any clocks have drifted beyond threshold."""
        # YOUR CODE HERE
        pass
    
    def synchronize_clocks(self):
        """Synchronize all clocks using Berkeley Algorithm."""
        # YOUR CODE HERE
        pass
    
    def simulate(self, duration=100, check_interval=10):
        """Run simulation for specified duration."""
        # YOUR CODE HERE
        pass

# Test your implementation
# monitor = ClockMonitor(num_servers=5, drift_threshold_ms=100)
# monitor.simulate(duration=100, check_interval=10)

### Exercise 3: Vector Clocks

**Task**: Implement Vector Clocks to capture causality between events.

**Vector Clock Rules**:
1. Each process maintains a vector of logical clocks (one per process)
2. On local event: increment own clock in vector
3. On send: increment own clock, attach vector to message
4. On receive: update vector to `max(local, received)` for each element, then increment own

**Why Vector Clocks?**
- Lamport timestamps can't distinguish concurrent events
- Vector clocks can detect: A happens-before B, B happens-before A, or A || B (concurrent)

In [ ]:
class VectorClock:
    """
    EXERCISE: Implement Vector Clocks.
    
    TODO: Complete this class
    """
    def __init__(self, process_id, num_processes):
        self.process_id = process_id
        self.num_processes = num_processes
        # Initialize vector with zeros
        # YOUR CODE HERE
        pass
    
    def local_event(self):
        """Process local event: increment own clock."""
        # YOUR CODE HERE
        pass
    
    def send_message(self):
        """Send message: increment own clock and return vector."""
        # YOUR CODE HERE
        pass
    
    def receive_message(self, received_vector):
        """Receive message: merge vectors and increment own clock."""
        # YOUR CODE HERE
        pass
    
    def happens_before(self, other_vector):
        """Check if self vector happens before other vector."""
        # YOUR CODE HERE
        pass
    
    def concurrent(self, other_vector):
        """Check if self vector is concurrent with other vector."""
        # YOUR CODE HERE
        pass

# Test your implementation
# P1 = VectorClock(process_id=0, num_processes=3)
# P2 = VectorClock(process_id=1, num_processes=3)
# P3 = VectorClock(process_id=2, num_processes=3)
#
# P1.local_event()
# msg = P1.send_message()
# P2.receive_message(msg)
# print(f"P1 vector: {P1.vector}")
# print(f"P2 vector: {P2.vector}")

---

## Summary

In this module, you learned about clock synchronization in distributed systems:

### Physical Clock Synchronization

✅ **Cristian's Algorithm**:
- Simple client-server approach
- Client synchronizes with accurate time server
- Accuracy depends on network delay symmetry

✅ **Berkeley Algorithm**:
- Fault-tolerant averaging approach
- No single point of failure
- Master coordinates, but uses average of all clocks

✅ **Network Time Protocol (NTP)**:
- Industry standard (not implemented here)
- Uses multiple time sources
- Filters outliers, handles network variability

### Logical Clock Synchronization

✅ **Lamport Timestamps**:
- Orders events without physical clocks
- Simple counter-based approach
- Guarantees: if A→B then timestamp(A) < timestamp(B)
- Cannot detect concurrent events

✅ **Vector Clocks** (Exercise):
- Captures causality completely
- Can detect concurrent events
- More overhead (N integers per process)

### Key Takeaways

1. **Perfect synchronization is impossible** in distributed systems (network delays are unpredictable)
2. **Physical clocks** are needed for wall-clock time (timestamps, expiration)
3. **Logical clocks** are sufficient for ordering events
4. **Trade-offs** exist between accuracy, overhead, and fault tolerance

### What's Next?

In **Module 05**, we'll explore:
- Distributed Operating Systems
- Process management in distributed systems
- Distributed file systems
- Protection and security

---

## Additional Resources

### Papers
- Lamport, L. (1978). "Time, Clocks, and the Ordering of Events in a Distributed System"
- Cristian, F. (1989). "Probabilistic Clock Synchronization"

### Tools
- [ntpd](https://www.ntp.org/) - Network Time Protocol daemon
- [Chrony](https://chrony.tuxfamily.org/) - Versatile NTP implementation
- [ptpd](https://github.com/ptpd/ptpd) - Precision Time Protocol daemon

---

**Congratulations!** You've completed Module 04: Clock Synchronization in Distributed Systems. 🎉